# Análise da taxa de aprovação das disciplinas
O objetivo desse notebook é realizar uma análise da taxa de aprovação das disciplinas da UFRN, por período.

**Perguntas orientadoras**:
- Qual o índice de aprovação da disciplina FMC2 em 2019.1?

# Os dados
Os dados que iremos utilizar foram retirados do site de [dados abertos da UFRN](http://dados.ufrn.br/). Os pacotes de dados foram:

- [Docentes](http://dados.ufrn.br/dataset/docentes)
- [Matriculas em Componentes](http://dados.ufrn.br/dataset/matriculas-componentes)
- [Turmas](http://dados.ufrn.br/dataset/turmas)

## Dependências
Para o download desses dados, iremos utilizar o pacote [odufrn-downloader](https://github.com/odufrn/odufrn-downloader).

In [ ]:
!pip install -U --user odufrn_downloader

## Importações

In [1]:
import numpy as np
import pandas as pd
from os import listdir
from odufrn_downloader import ODUFRNDownloader

## Download dos arquivos

In [2]:
# Objeto
ufrn_data = ODUFRNDownloader()
# Processo de download
ufrn_data.download_packages([
    'docentes', 'matriculas-componentes', 'turmas',
], path='data', dictionary=False)

Baixando Docentes...
Baixando Matrículas de 2019.1...
Baixando Matrículas de 2018.2...
Baixando Matrículas de 2018.1...
Baixando Matrículas de 2017.2...
Baixando Matrículas de 2017.1...
Baixando Matrículas de 2016.2...
Baixando Matrículas de 2016.1...
Baixando Matrículas de 2015.2...
Baixando Matrículas de 2015.1...
Baixando Matrículas de 2014.2...
Baixando Matrículas de 2014.1...
Baixando Matrículas de 2013.2...
Baixando Matrículas de 2013.1...
Baixando Matrículas de 2012.2...
Baixando Matrículas de 2011.2...
Baixando Matrículas de 2012.1...
Baixando Matrículas de 2011.1...
Baixando Matrículas de 2010.2...
Baixando Matrículas de 2010.1...
Baixando Matrículas de 2009.2...
Baixando Matrículas de 2009.1...
Baixando Turmas de 2018.1...
Baixando Turmas de 2018.2...
Baixando Turmas de 2017.1...
Baixando Turmas de 2017.2...
Baixando Turmas de 2016.1...
Baixando Turmas de 2016.2...
Baixando Turmas de 2015.1...
Baixando Turmas de 2015.2...
Baixando Turmas de 2014.1...
Baixando Turmas de 2014.2

## Leitura dos arquivos
Como temos muitos arquivos, não podemos nos dar ao luxo de alocar todos ele em memória, para isso iremos especificar os campos que queremos usar de cada pacote de dados.

**Docentes**:

| Campo | Tipo | Descrição | Obrigatório? |
| ----- | ---- | --------- | ------------ |
| siape | Inteiro | Matrícula SIAPE do docente | Sim |
| nome | Texto | Nome do docente | Sim |
| formacao | Texto | Denominação da formação/titulação do docente | Não |

**Turmas**:

| Campo | Tipo | Descrição | Obrigatório? |
| ----- | ---- | --------- | ------------ |
| id_turma | Inteiro | Identificador da turma | Sim |
| siape | Inteiro | Siape do docente interno (siape - caso seja um docente servidor da instituição). | Não |
| id_componente | Inteiro | Identificador do componente curricular da turma. | Sim |
| nivel_ensino | Texto | Nível de ensino da turma. | Sim |
| situacao_turma | Texto | Situação da turma. | Sim |

**Matrículas em Componentes**:

| Campo | Tipo | Descrição | Obrigatório? |
| ----- | ---- | --------- | ------------ |
| id_turma | Inteiro | Identificador da turma do discente. | Sim |
| id_discente | Texto | Identificador do discente da matrícula. | Sim |
| id_curso | Inteiro | Identificador do curso do discente. | Sim |
| media_final | Decimal | Média final do discente. | Sim |
| descricao | Texto | Situação da matrícula. | Sim |

In [8]:
# Campos que iremos usar de cada dataset
target_columns = {
    'docentes': [
        'siape',
        'nome',
        'formacao',
    ],
    'turmas': [
        'id_turma',
        'siape',
        'id_componente_curricular',
        'nivel_ensino',
        'situacao_turma',
    ],
    'matriculas-componentes': [
        'id_turma',
        'discente',
        'id_curso',
        'media_final',
        'descricao',
    ],
    
}

Realizamos o processo de filtragem dos campos e concatenação dos DataFrames. No caso de **turmas** e **matriculas-componentes**, iremos criar o campo *ano_periodo*. 

In [9]:
# Dicionário que conterá os datasets
dfs = {
    'docentes': [],
    'turmas': [],
    'matriculas-componentes': [],
}



# Percorremos as pastas com os datasets
for package in listdir('data'):
    # Percorremos cada arquivo da pasta
    for filename in listdir('data/'+package):
        # Caminho para o arquivo
        path_file = 'data/'+package+'/'+filename
        # Lemos o dataset
        df = pd.read_csv(path_file, sep=';', usecols=target_columns[package])
        # Adiciona a coluna para identificar os anos e períodos
        if package in ['turmas', 'matriculas-componentes']:
            df['ano_periodo'] = filename[-10:-4]
        # Adicionamos o dataframe a lista de dataframe
        dfs[package].append(df)
        
# Ao final, concatenamos todos os datasets, gerando um só para cada pacote
for key in dfs.keys():
    dfs[key] = pd.concat(dfs[key], ignore_index=True)

A função **[pandas.DataFrame.info](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.info.html)** nos fornecerá um sumário dos dados que temos. 

In [10]:
dfs['docentes'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2361 entries, 0 to 2360
Data columns (total 3 columns):
siape       2361 non-null int64
nome        2361 non-null object
formacao    2361 non-null object
dtypes: int64(1), object(2)
memory usage: 55.5+ KB


In [11]:
dfs['turmas'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 371632 entries, 0 to 371631
Data columns (total 6 columns):
id_turma                    371632 non-null int64
siape                       358806 non-null float64
id_componente_curricular    371632 non-null int64
nivel_ensino                371632 non-null object
situacao_turma              371632 non-null object
ano_periodo                 371632 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 17.0+ MB


In [13]:
dfs['matriculas-componentes'].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13079106 entries, 0 to 13079105
Data columns (total 6 columns):
id_turma       int64
discente       object
id_curso       float64
media_final    object
descricao      object
ano_periodo    object
dtypes: float64(1), int64(1), object(4)
memory usage: 598.7+ MB


# Calcular a taxa de aprovação das turmas

Primeiro iremos realizar um filtro nas turmas, pegando apenas as que são de graduação e foram consolidadas.

In [23]:
# Filtramos as turmas de graduação e turmas consolidadas
df_turmas_filter = dfs['turmas'][
    np.logical_and(
        dfs['turmas']['nivel_ensino'] == 'GRADUAÇÃO',
        dfs['turmas']['situacao_turma'] == 'CONSOLIDADA')
]

In [24]:
df_turmas_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 275534 entries, 74 to 371631
Data columns (total 6 columns):
id_turma                    275534 non-null int64
siape                       272762 non-null float64
id_componente_curricular    275534 non-null int64
nivel_ensino                275534 non-null object
situacao_turma              275534 non-null object
ano_periodo                 275534 non-null object
dtypes: float64(1), int64(2), object(3)
memory usage: 14.7+ MB


In [25]:
df_turmas_filter.head()

,id_turma,siape,id_componente_curricular,nivel_ensino,situacao_turma,ano_periodo
74,1160176,1149368.0,21743,GRADUAÇÃO,CONSOLIDADA,2011.1
75,1160177,1856399.0,22926,GRADUAÇÃO,CONSOLIDADA,2011.1
76,1160178,6346790.0,25562,GRADUAÇÃO,CONSOLIDADA,2011.1
77,1160179,346468.0,25562,GRADUAÇÃO,CONSOLIDADA,2011.1
78,1160179,6346790.0,25562,GRADUAÇÃO,CONSOLIDADA,2011.1


Agora vamos realizar o filtro nas matriculas, pegando as matriculas completaram a disciplina de alguma forma.

In [29]:
dfs['matriculas-componentes']['descricao'].unique()

array(['INDEFERIDO', 'APROVADO', 'TRANCADO', 'REPROVADO',
       'REPROVADO POR FALTAS', 'CANCELADO', 'EXCLUIDA', 'DESISTENCIA',
       'DISPENSADO', 'REPROVADO POR MÉDIA E POR FALTAS', 'MATRICULADO',
       'TRANSFERIDO', 'APROVADO POR NOTA', 'REPROVADO POR NOTA',
       'REPROVADO POR NOTA E FALTA', 'CUMPRIU', 'AGUARDANDO DEFERIMENTO',
       'EM ESPERA', 'INCORPORADO', 'REPROVADO EM TODO PERÍODO LETIVO'],
      dtype=object)

In [35]:
# Filtramos as matriculas que completaram a disciplina de alguma forma
df_matriculas_filter = dfs['matriculas-componentes'][
    dfs['matriculas-componentes']['descricao'].isin([
            'APROVADO',
            'APROVADO POR NOTA',
            'REPROVADO',
            'REPROVADO POR FALTAS',
            'REPROVADO POR MÉDIA E POR FALTAS',
            'REPROVADO POR NOTA',
            'REPROVADO POR NOTA E FALTA'
        ])
]

In [36]:
df_matriculas_filter.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 11237077 entries, 2 to 13079105
Data columns (total 6 columns):
id_turma       int64
discente       object
id_curso       float64
media_final    object
descricao      object
ano_periodo    object
dtypes: float64(1), int64(1), object(4)
memory usage: 600.1+ MB


In [37]:
df_matriculas_filter.head()

,id_turma,discente,id_curso,media_final,descricao,ano_periodo
2,57508432,bc3f5fa0b73af682c8e8f5ec1663028b,2000011.0,8.5,APROVADO,2009.1
3,57508432,bc3f5fa0b73af682c8e8f5ec1663028b,2000011.0,8.5,APROVADO,2009.1
4,57508432,bc3f5fa0b73af682c8e8f5ec1663028b,2000011.0,8.5,APROVADO,2009.1
7,57508432,a9bbcaaf5d35b4f71ffd224bb1fedbe5,2000011.0,8.2,APROVADO,2009.1
8,57508432,a9bbcaaf5d35b4f71ffd224bb1fedbe5,2000011.0,8.2,APROVADO,2009.1
